# Row Compression

Check out the data we're working with. All the parking tickets written in the City of Chicago from 2003 to 2016.

In [ ]:
USE ParkingTickets;
GO

SELECT TOP (100) *
FROM dbo.ParkingTickets;


For purposes of demo performance during a live session, let's copy the top 100K rows off into another table and work with that instead.

In [ ]:
SELECT TOP (100000) *
INTO dbo.ParkingTicketsMini
FROM dbo.ParkingTickets;


Run the following query to see the size of the ParkingTicketsMini table

In [ ]:
SELECT 
	s.name AS SchemaName, 
	t.name AS TableName, 
	i.index_id,
	LTRIM(LEFT(CONVERT(CHAR(25),CONVERT(MONEY,p.rows),1),22)) AS [rows],
	p.data_compression_desc,
	ROUND(CONVERT(FLOAT,MAX(a.total_pages))/128,3) AS Total_MB,
	ROUND(CONVERT(FLOAT,MAX(a.used_pages))/128,3) AS Used_MB
FROM sys.tables t
INNER JOIN sys.schemas s ON t.schema_id = s.schema_id
INNER JOIN sys.indexes i ON t.object_id = i.object_id
INNER JOIN sys.partitions p on t.object_id = p.object_id AND i.index_id = p.index_id
INNER JOIN sys.allocation_units a ON a.container_id = CASE WHEN a.type IN (1,3) THEN p.hobt_id WHEN a.type=2 THEN p.partition_id ELSE NULL END
WHERE s.name = 'dbo'
	AND t.name = 'ParkingTicketsMini'
GROUP BY s.name, t.name, i.index_id, p.rows, p.data_compression_desc


Apply Row Compression:

In [ ]:
ALTER TABLE dbo.ParkingTicketsMini REBUILD WITH (DATA_COMPRESSION = ROW);

Check how row compression affected the size:

In [ ]:
SELECT 
	s.name AS SchemaName, 
	t.name AS TableName, 
	i.index_id,
	LTRIM(LEFT(CONVERT(CHAR(25),CONVERT(MONEY,p.rows),1),22)) AS [rows],
	p.data_compression_desc,
	ROUND(CONVERT(FLOAT,MAX(a.total_pages))/128,3) AS Total_MB,
	ROUND(CONVERT(FLOAT,MAX(a.used_pages))/128,3) AS Used_MB
FROM sys.tables t
INNER JOIN sys.schemas s ON t.schema_id = s.schema_id
INNER JOIN sys.indexes i ON t.object_id = i.object_id
INNER JOIN sys.partitions p on t.object_id = p.object_id AND i.index_id = p.index_id
INNER JOIN sys.allocation_units a ON a.container_id = CASE WHEN a.type IN (1,3) THEN p.hobt_id WHEN a.type=2 THEN p.partition_id ELSE NULL END
WHERE s.name = 'dbo'
	AND t.name = 'ParkingTicketsMini'
GROUP BY s.name, t.name, i.index_id, p.rows, p.data_compression_desc

List out the pages that make up the table. Find the first one

In [ ]:
SELECT allocated_page_file_id, 
    allocated_page_page_id, 
    page_type_desc, 
    next_page_file_id, 
    next_page_page_id, 
    previous_page_file_id, 
    previous_page_page_id
FROM sys.dm_db_database_page_allocations (
    db_id('ParkingTickets'), -- database
    object_id('dbo.ParkingTicketsMini'), -- object
    0, -- index id
    1, -- partition number
    'DETAILED' -- mode {LIMITED|DETAILED}
)
WHERE page_type_desc = 'DATA_PAGE'
    AND previous_page_page_id IS NULL
    AND next_page_page_id IS NOT NULL

## Run this code in a separate window

```
DBCC TRACEON (3604);
GO
```
```
DBCC PAGE ('ParkingTickets', -- database
            3, -- file number of first page
            <pageNumber>, -- page number of first page
            3 -- show full details
            )
```

# Page Compression

Rebuild the table with page compression

In [ ]:
ALTER TABLE dbo.ParkingTicketsMini REBUILD WITH (DATA_COMPRESSION = PAGE);

Now see what page compression did to the size

In [ ]:
SELECT 
	s.name AS SchemaName, 
	t.name AS TableName, 
	i.index_id,
	LTRIM(LEFT(CONVERT(CHAR(25),CONVERT(MONEY,p.rows),1),22)) AS [rows],
	p.data_compression_desc,
	ROUND(CONVERT(FLOAT,MAX(a.total_pages))/128,3) AS Total_MB,
	ROUND(CONVERT(FLOAT,MAX(a.used_pages))/128,3) AS Used_MB
FROM sys.tables t
INNER JOIN sys.schemas s ON t.schema_id = s.schema_id
INNER JOIN sys.indexes i ON t.object_id = i.object_id
INNER JOIN sys.partitions p on t.object_id = p.object_id AND i.index_id = p.index_id
INNER JOIN sys.allocation_units a ON a.container_id = CASE WHEN a.type IN (1,3) THEN p.hobt_id WHEN a.type=2 THEN p.partition_id ELSE NULL END
WHERE s.name = 'dbo'
	AND t.name = 'ParkingTicketsMini'
GROUP BY s.name, t.name, i.index_id, p.rows, p.data_compression_desc

Find the first page of the table now

In [ ]:
SELECT allocated_page_file_id, 
    allocated_page_page_id, 
    page_type_desc, 
    next_page_file_id, 
    next_page_page_id, 
    previous_page_file_id, 
    previous_page_page_id
FROM sys.dm_db_database_page_allocations (
    db_id('ParkingTickets'),
    object_id('dbo.ParkingTicketsMini'),
    NULL,
    NULL,
    'DETAILED'
)
WHERE page_type_desc = 'DATA_PAGE'
    AND previous_page_page_id IS NULL
    AND next_page_page_id IS NOT NULL

Run DBCC PAGE code from before again in a separate window (with new page number)

You can see whether a partition is compressed or not by checking sys.partitions

In [ ]:
SELECT * from sys.partitions;

Filter out system objects:

In [ ]:
SELECT p.partition_id, 
	s.name AS SchemaName,
	o.name AS ObjectName,
	i.name AS IndexName,
	p.partition_number,
	p.hobt_id,
	p.rows,
	p.filestream_filegroup_id,
	p.data_compression_desc
FROM sys.partitions p
INNER JOIN sys.objects o ON p.object_id = o.object_id
INNER JOIN sys.indexes i ON p.object_id = i.object_id and p.index_id = i.index_id
INNER JOIN sys.schemas s ON o.schema_id = s.schema_id
WHERE s.name NOT IN ('sys','INFORMATION_SCHEMA') -- filter out system schemas (system tables can't be compressed)
ORDER BY SchemaName, ObjectName, IndexName, partition_number;

<span style="font-size: 14px;">Is this database using any features that may require Enterprise Edition? (Data compression no longer does but still shows in this listing)</span>

In [ ]:
select * from sys.dm_db_persisted_sku_features;

Estimate how much savings can be attained with either Row or Page compression:

In [ ]:
EXEC sp_estimate_data_compression_savings 
    'dbo', -- schema name
    'ParkingTicketsMini', -- object name
    NULL, -- index id
    NULL, -- partition number
    ROW; -- {NONE|ROW|PAGE|COLUMNSTORE|COLUMNSTORE_ARCHIVE}

In [ ]:
EXEC sp_estimate_data_compression_savings 'dbo', 'ParkingTicketsMini', NULL, NULL, COLUMNSTORE;

In [ ]:
EXEC sp_estimate_data_compression_savings 'dbo', 'ParkingTicketsMini', NULL, NULL, COLUMNSTORE_ARCHIVE;

Cleanup

In [ ]:
DROP TABLE IF EXISTS dbo.ParkingTicketsMini;